# Anna KaRNNa

In this notebook, we'll build a character-wise RNN trained on Anna Karenina, one of my all-time favorite books. It'll be able to generate new text based on the text from the book.

This network is based off of Andrej Karpathy's [post on RNNs](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) and [implementation in Torch](https://github.com/karpathy/char-rnn). Also, some information [here at r2rt](http://r2rt.com/recurrent-neural-networks-in-tensorflow-ii.html) and from [Sherjil Ozair](https://github.com/sherjilozair/char-rnn-tensorflow) on GitHub. Below is the general architecture of the character-wise RNN.

<img src="assets/charseq.jpeg" width="500">

In [1]:
import time
from collections import namedtuple

import numpy as np
import tensorflow as tf

First we'll load the text file and convert it into integers for our network to use. Here I'm creating a couple dictionaries to convert the characters to and from integers. Encoding the characters as integers makes it easier to use as input in the network.

In [2]:
with open('all.js', 'r') as f:
    text=f.read()
vocab = sorted(set(text))
vocab_to_int = {c: i for i, c in enumerate(vocab)}
int_to_vocab = dict(enumerate(vocab))
encoded = np.array([vocab_to_int[c] for c in text], dtype=np.int32)

Let's check out the first 100 characters, make sure everything is peachy. According to the [American Book Review](http://americanbookreview.org/100bestlines.asp), this is the 6th best first line of a book ever.

In [3]:
text[:100]

'// Originally from narwhal.js (http://narwhaljs.org)\n// Copyright (c) 2009 Thomas Robinson <280north'

And we can see the characters encoded as integers.

In [4]:
encoded[:100]

array([17, 17,  2, 49, 84, 75, 73, 75, 80, 67, 78, 78, 91,  2, 72, 84, 81,
       79,  2, 80, 67, 84, 89, 74, 67, 78, 16, 76, 85,  2, 10, 74, 86, 86,
       82, 28, 17, 17, 80, 67, 84, 89, 74, 67, 78, 76, 85, 16, 81, 84, 73,
       11,  1, 17, 17,  2, 37, 81, 82, 91, 84, 75, 73, 74, 86,  2, 10, 69,
       11,  2, 20, 18, 18, 27,  2, 54, 74, 81, 79, 67, 85,  2, 52, 81, 68,
       75, 80, 85, 81, 80,  2, 30, 20, 26, 18, 80, 81, 84, 86, 74],
      dtype=int32)

Since the network is working with individual characters, it's similar to a classification problem in which we are trying to predict the next character from the previous text.  Here's how many 'classes' our network has to pick from.

In [5]:
len(vocab)

114

## Making training mini-batches

Here is where we'll make our mini-batches for training. Remember that we want our batches to be multiple sequences of some desired number of sequence steps. Considering a simple example, our batches would look like this:

<img src="assets/sequence_batching@1x.png" width=500px>


<br>

We start with our text encoded as integers in one long array in `encoded`. Let's create a function that will give us an iterator for our batches. I like using [generator functions](https://jeffknupp.com/blog/2013/04/07/improve-your-python-yield-and-generators-explained/) to do this. Then we can pass `encoded` into this function and get our batch generator.

The first thing we need to do is discard some of the text so we only have completely full batches. Each batch contains $N \times M$ characters, where $N$ is the batch size (the number of sequences) and $M$ is the number of steps. Then, to get the total number of batches, $K$, we can make from the array `arr`, you divide the length of `arr` by the number of characters per batch. Once you know the number of batches, you can get the total number of characters to keep from `arr`, $N * M * K$.

After that, we need to split `arr` into $N$ sequences. You can do this using `arr.reshape(size)` where `size` is a tuple containing the dimensions sizes of the reshaped array. We know we want $N$ sequences (`batch_size` below), let's make that the size of the first dimension. For the second dimension, you can use `-1` as a placeholder in the size, it'll fill up the array with the appropriate data for you. After this, you should have an array that is $N \times (M * K)$.

Now that we have this array, we can iterate through it to get our batches. The idea is each batch is a $N \times M$ window on the $N \times (M * K)$ array. For each subsequent batch, the window moves over by `n_steps`. We also want to create both the input and target arrays. Remember that the targets are the inputs shifted over one character. 

The way I like to do this window is use `range` to take steps of size `n_steps` from $0$ to `arr.shape[1]`, the total number of steps in each sequence. That way, the integers you get from `range` always point to the start of a batch, and each window is `n_steps` wide.

> **Exercise:** Write the code for creating batches in the function below. The exercises in this notebook _will not be easy_. I've provided a notebook with solutions alongside this notebook. If you get stuck, checkout the solutions. The most important thing is that you don't copy and paste the code into here, **type out the solution code yourself.**

In [6]:
def get_batches(arr, batch_size, n_steps):
    '''Create a generator that returns batches of size
       batch_size x n_steps from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       n_steps: Number of sequence steps per batch
    '''
    # Get the number of characters per batch and number of batches we can make
    characters_per_batch = batch_size * n_steps
    n_batches = len(arr) // characters_per_batch
    
    # Keep only enough characters to make full batches
    arr = arr[:n_batches * characters_per_batch]
    
    # Reshape into batch_size rows
    arr = arr.reshape((batch_size, -1))
    
    for n in range(0, arr.shape[1], n_steps):
        # The features
        x = arr[:, n:n+n_steps]
        # The targets, shifted by one
        y = np.zeros_like(x)
        y[:, :-1], y[:, -1] = x[:, 1:], x[:, 0]
        yield x, y

Now I'll make my data sets and we can check out what's going on here. Here I'm going to use a batch size of 10 and 50 sequence steps.

In [7]:
batches = get_batches(encoded, 10, 50)
x, y = next(batches)

In [8]:
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[17 17  2 49 84 75 73 75 80 67]
 [86 86 71 84 29  1  1 17 17  2]
 [ 1  2  2  2  2 95  2 71 78 85]
 [ 2  2  2 86 67 84 73 71 86 16]
 [ 9 84 71 67 78 82 67 86 74 37]
 [ 2  9  2 13  2 76 85 81 80 50]
 [50 39 10  9 81 82 86 75 81 80]
 [ 2 17 17  2 54 74 75 85  2 82]
 [ 2  2  2  2  2 86 84 91  2 93]
 [84 67 82  2 31  2 85 81 69 77]]

y
 [[17  2 49 84 75 73 75 80 67 78]
 [86 71 84 29  1  1 17 17  2 36]
 [ 2  2  2  2 95  2 71 78 85 71]
 [ 2  2 86 67 84 73 71 86 16 69]
 [84 71 67 78 82 67 86 74 37 67]
 [ 9  2 13  2 76 85 81 80 50 67]
 [39 10  9 81 82 86 75 81 80 85]
 [17 17  2 54 74 75 85  2 82 67]
 [ 2  2  2  2 86 84 91  2 93  1]
 [67 82  2 31  2 85 81 69 77 71]]


If you implemented `get_batches` correctly, the above output should look something like 
```
x
 [[55 63 69 22  6 76 45  5 16 35]
 [ 5 69  1  5 12 52  6  5 56 52]
 [48 29 12 61 35 35  8 64 76 78]
 [12  5 24 39 45 29 12 56  5 63]
 [ 5 29  6  5 29 78 28  5 78 29]
 [ 5 13  6  5 36 69 78 35 52 12]
 [63 76 12  5 18 52  1 76  5 58]
 [34  5 73 39  6  5 12 52 36  5]
 [ 6  5 29 78 12 79  6 61  5 59]
 [ 5 78 69 29 24  5  6 52  5 63]]

y
 [[63 69 22  6 76 45  5 16 35 35]
 [69  1  5 12 52  6  5 56 52 29]
 [29 12 61 35 35  8 64 76 78 28]
 [ 5 24 39 45 29 12 56  5 63 29]
 [29  6  5 29 78 28  5 78 29 45]
 [13  6  5 36 69 78 35 52 12 43]
 [76 12  5 18 52  1 76  5 58 52]
 [ 5 73 39  6  5 12 52 36  5 78]
 [ 5 29 78 12 79  6 61  5 59 63]
 [78 69 29 24  5  6 52  5 63 76]]
 ```
 although the exact numbers will be different. Check to make sure the data is shifted over one step for `y`.

## Building the model

Below is where you'll build the network. We'll break it up into parts so it's easier to reason about each bit. Then we can connect them up into the whole network.

<img src="assets/charRNN.png" width=500px>


### Inputs

First off we'll create our input placeholders. As usual we need placeholders for the training data and the targets. We'll also create a placeholder for dropout layers called `keep_prob`. This will be a scalar, that is a 0-D tensor. To make a scalar, you create a placeholder without giving it a size.

> **Exercise:** Create the input placeholders in the function below.

In [9]:
def build_inputs(batch_size, num_steps):
    ''' Define placeholders for inputs, targets, and dropout 
    
        Arguments
        ---------
        batch_size: Batch size, number of sequences per batch
        num_steps: Number of sequence steps in a batch
        
    '''
    # Declare placeholders we'll feed into the graph
    inputs = tf.placeholder(tf.int32, shape=(batch_size, num_steps), name="inputs")
    targets = tf.placeholder(tf.int32, shape=(batch_size, num_steps), name="targets")
    
    # Keep probability placeholder for drop out layers
    keep_prob = tf.placeholder(tf.float32, name="keep_prob")
    
    return inputs, targets, keep_prob

### LSTM Cell

Here we will create the LSTM cell we'll use in the hidden layer. We'll use this cell as a building block for the RNN. So we aren't actually defining the RNN here, just the type of cell we'll use in the hidden layer.

We first create a basic LSTM cell with

```python
lstm = tf.contrib.rnn.BasicLSTMCell(num_units)
```

where `num_units` is the number of units in the hidden layers in the cell. Then we can add dropout by wrapping it with 

```python
tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
```
You pass in a cell and it will automatically add dropout to the inputs or outputs. Finally, we can stack up the LSTM cells into layers with [`tf.contrib.rnn.MultiRNNCell`](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/contrib/rnn/MultiRNNCell). With this, you pass in a list of cells and it will send the output of one cell into the next cell. Previously with TensorFlow 1.0, you could do this

```python
tf.contrib.rnn.MultiRNNCell([cell]*num_layers)
```

This might look a little weird if you know Python well because this will create a list of the same `cell` object. However, TensorFlow 1.0 will create different weight matrices for all `cell` objects. But, starting with TensorFlow 1.1 you actually need to create new cell objects in the list. To get it to work in TensorFlow 1.1, it should look like

```python
def build_cell(num_units, keep_prob):
    lstm = tf.contrib.rnn.BasicLSTMCell(num_units)
    drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
    
    return drop
    
tf.contrib.rnn.MultiRNNCell([build_cell(num_units, keep_prob) for _ in range(num_layers)])
```

Even though this is actually multiple LSTM cells stacked on each other, you can treat the multiple layers as one cell.

We also need to create an initial cell state of all zeros. This can be done like so

```python
initial_state = cell.zero_state(batch_size, tf.float32)
```

Below, we implement the `build_lstm` function to create these LSTM cells and the initial state.

In [10]:
def build_lstm(lstm_size, num_layers, batch_size, keep_prob):
    ''' Build LSTM cell.
    
        Arguments
        ---------
        keep_prob: Scalar tensor (tf.placeholder) for the dropout keep probability
        lstm_size: Size of the hidden layers in the LSTM cells
        num_layers: Number of LSTM layers
        batch_size: Batch size

    '''
    ### Build the LSTM Cell
    # Use a basic LSTM cell
    def build_cell(lstm_size, keep_prob):
        lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)

        # Add dropout to the cell outputs
        drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        return drop
    
    # Stack up multiple LSTM layers, for deep learning
    cell = tf.contrib.rnn.MultiRNNCell([build_cell(lstm_size, keep_prob) for _ in range(num_layers)])
    initial_state = cell.zero_state(batch_size, tf.float32)
    
    return cell, initial_state

### RNN Output

Here we'll create the output layer. We need to connect the output of the RNN cells to a full connected layer with a softmax output. The softmax output gives us a probability distribution we can use to predict the next character, so we want this layer to have size $C$, the number of classes/characters we have in our text.

If our input has batch size $N$, number of steps $M$, and the hidden layer has $L$ hidden units, then the output is a 3D tensor with size $N \times M \times L$. The output of each LSTM cell has size $L$, we have $M$ of them, one for each sequence step, and we have $N$ sequences. So the total size is $N \times M \times L$. 

We are using the same fully connected layer, the same weights, for each of the outputs. Then, to make things easier, we should reshape the outputs into a 2D tensor with shape $(M * N) \times L$. That is, one row for each sequence and step, where the values of each row are the output from the LSTM cells. We get the LSTM output as a list, `lstm_output`. First we need to concatenate this whole list into one array with [`tf.concat`](https://www.tensorflow.org/api_docs/python/tf/concat). Then, reshape it (with `tf.reshape`) to size $(M * N) \times L$.

One we have the outputs reshaped, we can do the matrix multiplication with the weights. We need to wrap the weight and bias variables in a variable scope with `tf.variable_scope(scope_name)` because there are weights being created in the LSTM cells. TensorFlow will throw an error if the weights created here have the same names as the weights created in the LSTM cells, which they will be default. To avoid this, we wrap the variables in a variable scope so we can give them unique names.

> **Exercise:** Implement the output layer in the function below.

In [11]:
def build_output(lstm_output, in_size, out_size):
    ''' Build a softmax layer, return the softmax output and logits.
    
        Arguments
        ---------
        
        lstm_output: List of output tensors from the LSTM layer
        in_size: Size of the input tensor, for example, size of the LSTM cells
        out_size: Size of this softmax layer
    
    '''

    # Reshape output so it's a bunch of rows, one row for each step for each sequence.
    # Concatenate lstm_output over axis 1 (the columns)
    seq_output = tf.concat(lstm_output, axis=1)
    # Reshape seq_output to a 2D tensor with lstm_size columns
    x = tf.reshape(seq_output, [-1, in_size])
    
    # Connect the RNN outputs to a softmax layer
    with tf.variable_scope('softmax'):
        # Create the weight and bias variables here
        softmax_w = tf.Variable(tf.truncated_normal((in_size, out_size), stddev=0.1))
        softmax_b = tf.Variable(tf.zeros(out_size))
    
    # Since output is a bunch of rows of RNN cell outputs, logits will be a bunch
    # of rows of logit outputs, one for each step and sequence
    logits = tf.matmul(x, softmax_w) + softmax_b
    
    # Use softmax to get the probabilities for predicted characters
    out = tf.nn.softmax(logits, name="predictions")
    
    return out, logits

### Training loss

Next up is the training loss. We get the logits and targets and calculate the softmax cross-entropy loss. First we need to one-hot encode the targets, we're getting them as encoded characters. Then, reshape the one-hot targets so it's a 2D tensor with size $(M*N) \times C$ where $C$ is the number of classes/characters we have. Remember that we reshaped the LSTM outputs and ran them through a fully connected layer with $C$ units. So our logits will also have size $(M*N) \times C$.

Then we run the logits and targets through `tf.nn.softmax_cross_entropy_with_logits` and find the mean to get the loss.

>**Exercise:** Implement the loss calculation in the function below.

In [12]:
def build_loss(logits, targets, lstm_size, num_classes):
    ''' Calculate the loss from the logits and the targets.
    
        Arguments
        ---------
        logits: Logits from final fully connected layer
        targets: Targets for supervised learning
        lstm_size: Number of LSTM hidden units
        num_classes: Number of classes in targets
        
    '''
    
    # One-hot encode targets and reshape to match logits, one row per sequence per step
    y_one_hot = tf.one_hot(targets, num_classes)
    y_reshaped = tf.reshape(y_one_hot, logits.get_shape())
    
    # Softmax cross entropy loss
    loss = tf.nn.softmax_cross_entropy_with_logits(labels=y_reshaped, logits=logits)
    loss = tf.reduce_mean(loss)
    
    return loss

### Optimizer

Here we build the optimizer. Normal RNNs have have issues gradients exploding and disappearing. LSTMs fix the disappearance problem, but the gradients can still grow without bound. To fix this, we can clip the gradients above some threshold. That is, if a gradient is larger than that threshold, we set it to the threshold. This will ensure the gradients never grow overly large. Then we use an AdamOptimizer for the learning step.

In [13]:
def build_optimizer(loss, learning_rate, grad_clip):
    ''' Build optmizer for training, using gradient clipping.
    
        Arguments:
        loss: Network loss
        learning_rate: Learning rate for optimizer
    
    '''
    
    # Optimizer for training, using gradient clipping to control exploding gradients
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), grad_clip)
    train_op = tf.train.AdamOptimizer(learning_rate)
    optimizer = train_op.apply_gradients(zip(grads, tvars))
    
    return optimizer

### Build the network

Now we can put all the pieces together and build a class for the network. To actually run data through the LSTM cells, we will use [`tf.nn.dynamic_rnn`](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/nn/dynamic_rnn). This function will pass the hidden and cell states across LSTM cells appropriately for us. It returns the outputs for each LSTM cell at each step for each sequence in the mini-batch. It also gives us the final LSTM state. We want to save this state as `final_state` so we can pass it to the first LSTM cell in the the next mini-batch run. For `tf.nn.dynamic_rnn`, we pass in the cell and initial state we get from `build_lstm`, as well as our input sequences. Also, we need to one-hot encode the inputs before going into the RNN. 

> **Exercise:** Use the functions you've implemented previously and `tf.nn.dynamic_rnn` to build the network.

In [14]:
class CharRNN:
    
    def __init__(self, num_classes, batch_size=64, num_steps=50, 
                       lstm_size=128, num_layers=2, learning_rate=0.001, 
                       grad_clip=5, sampling=False):
    
        # When we're using this network for sampling later, we'll be passing in
        # one character at a time, so providing an option for that
        if sampling == True:
            batch_size, num_steps = 1, 1
        else:
            batch_size, num_steps = batch_size, num_steps

        tf.reset_default_graph()
        
        # Build the input placeholder tensors
        self.inputs, self.targets, self.keep_prob = build_inputs(batch_size, num_steps)

        # Build the LSTM cell
        cell, self.initial_state = build_lstm(lstm_size, num_layers, batch_size, self.keep_prob)

        ### Run the data through the RNN layers
        # First, one-hot encode the input tokens
        x_one_hot = tf.one_hot(self.inputs, num_classes)
        
        # Run each sequence step through the RNN with tf.nn.dynamic_rnn 
        outputs, state = tf.nn.dynamic_rnn(cell, x_one_hot, initial_state=self.initial_state)
        self.final_state = state
        
        # Get softmax predictions and logits
        self.prediction, self.logits = build_output(outputs, lstm_size, num_classes)
        
        # Loss and optimizer (with gradient clipping)
        self.loss =  build_loss(self.logits, self.targets, lstm_size, num_classes)
        self.optimizer = build_optimizer(self.loss, learning_rate, grad_clip)

## Hyperparameters

Here are the hyperparameters for the network.

* `batch_size` - Number of sequences running through the network in one pass.
* `num_steps` - Number of characters in the sequence the network is trained on. Larger is better typically, the network will learn more long range dependencies. But it takes longer to train. 100 is typically a good number here.
* `lstm_size` - The number of units in the hidden layers.
* `num_layers` - Number of hidden LSTM layers to use
* `learning_rate` - Learning rate for training
* `keep_prob` - The dropout keep probability when training. If you're network is overfitting, try decreasing this.

Here's some good advice from Andrej Karpathy on training the network. I'm going to copy it in here for your benefit, but also link to [where it originally came from](https://github.com/karpathy/char-rnn#tips-and-tricks).

> ## Tips and Tricks

>### Monitoring Validation Loss vs. Training Loss
>If you're somewhat new to Machine Learning or Neural Networks it can take a bit of expertise to get good models. The most important quantity to keep track of is the difference between your training loss (printed during training) and the validation loss (printed once in a while when the RNN is run on the validation data (by default every 1000 iterations)). In particular:

> - If your training loss is much lower than validation loss then this means the network might be **overfitting**. Solutions to this are to decrease your network size, or to increase dropout. For example you could try dropout of 0.5 and so on.
> - If your training/validation loss are about equal then your model is **underfitting**. Increase the size of your model (either number of layers or the raw number of neurons per layer)

> ### Approximate number of parameters

> The two most important parameters that control the model are `lstm_size` and `num_layers`. I would advise that you always use `num_layers` of either 2/3. The `lstm_size` can be adjusted based on how much data you have. The two important quantities to keep track of here are:

> - The number of parameters in your model. This is printed when you start training.
> - The size of your dataset. 1MB file is approximately 1 million characters.

>These two should be about the same order of magnitude. It's a little tricky to tell. Here are some examples:

> - I have a 100MB dataset and I'm using the default parameter settings (which currently print 150K parameters). My data size is significantly larger (100 mil >> 0.15 mil), so I expect to heavily underfit. I am thinking I can comfortably afford to make `lstm_size` larger.
> - I have a 10MB dataset and running a 10 million parameter model. I'm slightly nervous and I'm carefully monitoring my validation loss. If it's larger than my training loss then I may want to try to increase dropout a bit and see if that helps the validation loss.

> ### Best models strategy

>The winning strategy to obtaining very good models (if you have the compute time) is to always err on making the network larger (as large as you're willing to wait for it to compute) and then try different dropout values (between 0,1). Whatever model has the best validation performance (the loss, written in the checkpoint filename, low is good) is the one you should use in the end.

>It is very common in deep learning to run many different models with many different hyperparameter settings, and in the end take whatever checkpoint gave the best validation performance.

>By the way, the size of your training and validation splits are also parameters. Make sure you have a decent amount of data in your validation set or otherwise the validation performance will be noisy and not very informative.

In [15]:
batch_size = 10         # Sequences per batch
num_steps = 50          # Number of sequence steps per batch
lstm_size = 128         # Size of hidden layers in LSTMs
num_layers = 2          # Number of LSTM layers
learning_rate = 0.01    # Learning rate
keep_prob = 0.5         # Dropout keep probability

## Time for training

This is typical training code, passing inputs and targets into the network, then running the optimizer. Here we also get back the final LSTM state for the mini-batch. Then, we pass that state back into the network so the next batch can continue the state from the previous batch. And every so often (set by `save_every_n`) I save a checkpoint.

Here I'm saving checkpoints with the format

`i{iteration number}_l{# hidden layer units}.ckpt`

> **Exercise:** Set the hyperparameters above to train the network. Watch the training loss, it should be consistently dropping. Also, I highly advise running this on a GPU.

In [21]:
epochs = 50
# Print losses every N interations
print_every_n = 50

# Save every N iterations
save_every_n = 200

model = CharRNN(len(vocab), batch_size=batch_size, num_steps=num_steps,
                lstm_size=lstm_size, num_layers=num_layers, 
                learning_rate=learning_rate)

saver = tf.train.Saver(max_to_keep=100)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # Use the line below to load a checkpoint and resume training
    #saver.restore(sess, 'checkpoints/______.ckpt')
    counter = 0
    for e in range(epochs):
        # Train network
        new_state = sess.run(model.initial_state)
        loss = 0
        for x, y in get_batches(encoded, batch_size, num_steps):
            counter += 1
            start = time.time()
            feed = {model.inputs: x,
                    model.targets: y,
                    model.keep_prob: keep_prob,
                    model.initial_state: new_state}
            batch_loss, new_state, _ = sess.run([model.loss, 
                                                 model.final_state, 
                                                 model.optimizer], 
                                                 feed_dict=feed)
            if (counter % print_every_n == 0):
                end = time.time()
                print('Epoch: {}/{}... '.format(e+1, epochs),
                      'Training Step: {}... '.format(counter),
                      'Training loss: {:.4f}... '.format(batch_loss),
                      '{:.4f} sec/batch'.format((end-start)))
        
            if (counter % save_every_n == 0):
                saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))
    
    saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))

Epoch: 1/50...  Training Step: 50...  Training loss: 3.5526...  0.0670 sec/batch
Epoch: 1/50...  Training Step: 100...  Training loss: 3.2755...  0.0654 sec/batch
Epoch: 1/50...  Training Step: 150...  Training loss: 3.3693...  0.0738 sec/batch
Epoch: 1/50...  Training Step: 200...  Training loss: 3.2177...  0.0655 sec/batch
Epoch: 1/50...  Training Step: 250...  Training loss: 3.5641...  0.0679 sec/batch
Epoch: 1/50...  Training Step: 300...  Training loss: 3.6032...  0.0731 sec/batch
Epoch: 1/50...  Training Step: 350...  Training loss: 3.3525...  0.0657 sec/batch
Epoch: 1/50...  Training Step: 400...  Training loss: 3.3720...  0.0646 sec/batch
Epoch: 1/50...  Training Step: 450...  Training loss: 3.1987...  0.0662 sec/batch
Epoch: 1/50...  Training Step: 500...  Training loss: 3.3812...  0.0721 sec/batch
Epoch: 1/50...  Training Step: 550...  Training loss: 3.4149...  0.0660 sec/batch
Epoch: 1/50...  Training Step: 600...  Training loss: 3.2738...  0.0653 sec/batch
Epoch: 1/50...  T

Epoch: 2/50...  Training Step: 5000...  Training loss: 1.7238...  0.0678 sec/batch
Epoch: 2/50...  Training Step: 5050...  Training loss: 1.5913...  0.0664 sec/batch
Epoch: 2/50...  Training Step: 5100...  Training loss: 1.5395...  0.0742 sec/batch
Epoch: 2/50...  Training Step: 5150...  Training loss: 1.8313...  0.0753 sec/batch
Epoch: 2/50...  Training Step: 5200...  Training loss: 1.8871...  0.0756 sec/batch
Epoch: 2/50...  Training Step: 5250...  Training loss: 1.7929...  0.0658 sec/batch
Epoch: 2/50...  Training Step: 5300...  Training loss: 1.7814...  0.0688 sec/batch
Epoch: 2/50...  Training Step: 5350...  Training loss: 2.2672...  0.0661 sec/batch
Epoch: 2/50...  Training Step: 5400...  Training loss: 1.6756...  0.0658 sec/batch
Epoch: 2/50...  Training Step: 5450...  Training loss: 2.1413...  0.0667 sec/batch
Epoch: 2/50...  Training Step: 5500...  Training loss: 1.6251...  0.1096 sec/batch
Epoch: 2/50...  Training Step: 5550...  Training loss: 1.9457...  0.0651 sec/batch
Epoc

Epoch: 4/50...  Training Step: 9950...  Training loss: 1.6232...  0.0660 sec/batch
Epoch: 4/50...  Training Step: 10000...  Training loss: 1.6825...  0.0734 sec/batch
Epoch: 4/50...  Training Step: 10050...  Training loss: 1.8372...  0.0652 sec/batch
Epoch: 4/50...  Training Step: 10100...  Training loss: 1.5281...  0.0685 sec/batch
Epoch: 4/50...  Training Step: 10150...  Training loss: 1.5859...  0.0653 sec/batch
Epoch: 4/50...  Training Step: 10200...  Training loss: 1.6982...  0.0760 sec/batch
Epoch: 4/50...  Training Step: 10250...  Training loss: 1.6838...  0.0675 sec/batch
Epoch: 4/50...  Training Step: 10300...  Training loss: 1.8057...  0.0719 sec/batch
Epoch: 4/50...  Training Step: 10350...  Training loss: 1.6590...  0.0659 sec/batch
Epoch: 4/50...  Training Step: 10400...  Training loss: 1.6034...  0.0649 sec/batch
Epoch: 4/50...  Training Step: 10450...  Training loss: 1.5164...  0.0664 sec/batch
Epoch: 4/50...  Training Step: 10500...  Training loss: 1.7493...  0.0682 sec

Epoch: 6/50...  Training Step: 14850...  Training loss: 1.7770...  0.0648 sec/batch
Epoch: 6/50...  Training Step: 14900...  Training loss: 1.7648...  0.0656 sec/batch
Epoch: 6/50...  Training Step: 14950...  Training loss: 1.5990...  0.0735 sec/batch
Epoch: 6/50...  Training Step: 15000...  Training loss: 1.5164...  0.0657 sec/batch
Epoch: 6/50...  Training Step: 15050...  Training loss: 1.6616...  0.0661 sec/batch
Epoch: 6/50...  Training Step: 15100...  Training loss: 1.5709...  0.0705 sec/batch
Epoch: 6/50...  Training Step: 15150...  Training loss: 1.5164...  0.0671 sec/batch
Epoch: 6/50...  Training Step: 15200...  Training loss: 1.6316...  0.0732 sec/batch
Epoch: 6/50...  Training Step: 15250...  Training loss: 1.7281...  0.0643 sec/batch
Epoch: 6/50...  Training Step: 15300...  Training loss: 1.7996...  0.0659 sec/batch
Epoch: 6/50...  Training Step: 15350...  Training loss: 1.7133...  0.0727 sec/batch
Epoch: 6/50...  Training Step: 15400...  Training loss: 1.5567...  0.0659 se

Epoch: 8/50...  Training Step: 19750...  Training loss: 1.8114...  0.0662 sec/batch
Epoch: 8/50...  Training Step: 19800...  Training loss: 1.4768...  0.0656 sec/batch
Epoch: 8/50...  Training Step: 19850...  Training loss: 1.3318...  0.0709 sec/batch
Epoch: 8/50...  Training Step: 19900...  Training loss: 1.5250...  0.0741 sec/batch
Epoch: 8/50...  Training Step: 19950...  Training loss: 1.6905...  0.0662 sec/batch
Epoch: 8/50...  Training Step: 20000...  Training loss: 1.7759...  0.0646 sec/batch
Epoch: 8/50...  Training Step: 20050...  Training loss: 1.6739...  0.0691 sec/batch
Epoch: 8/50...  Training Step: 20100...  Training loss: 1.5381...  0.0658 sec/batch
Epoch: 8/50...  Training Step: 20150...  Training loss: 1.6587...  0.0684 sec/batch
Epoch: 8/50...  Training Step: 20200...  Training loss: 1.6571...  0.0735 sec/batch
Epoch: 8/50...  Training Step: 20250...  Training loss: 1.4387...  0.0649 sec/batch
Epoch: 8/50...  Training Step: 20300...  Training loss: 1.6657...  0.0696 se

...

Epoch: 44/50...  Training Step: 121400...  Training loss: 1.3393...  0.0658 sec/batch
Epoch: 44/50...  Training Step: 121450...  Training loss: 1.1914...  0.0763 sec/batch
Epoch: 44/50...  Training Step: 121500...  Training loss: 1.1763...  0.0669 sec/batch
Epoch: 44/50...  Training Step: 121550...  Training loss: 1.3107...  0.0673 sec/batch
Epoch: 44/50...  Training Step: 121600...  Training loss: 1.3610...  0.0709 sec/batch
Epoch: 44/50...  Training Step: 121650...  Training loss: 1.2871...  0.0669 sec/batch
Epoch: 44/50...  Training Step: 121700...  Training loss: 1.4786...  0.0648 sec/batch
Epoch: 44/50...  Training Step: 121750...  Training loss: 1.2489...  0.0649 sec/batch
Epoch: 44/50...  Training Step: 121800...  Training loss: 1.2993...  0.0665 sec/batch
Epoch: 44/50...  Training Step: 121850...  Training loss: 1.1139...  0.0663 sec/batch
Epoch: 44/50...  Training Step: 121900...  Training loss: 1.5181...  0.0707 sec/batch
Epoch: 44/50...  Training Step: 121950...  Training lo

Epoch: 45/50...  Training Step: 126200...  Training loss: 1.5108...  0.0651 sec/batch
Epoch: 45/50...  Training Step: 126250...  Training loss: 1.0968...  0.0677 sec/batch
Epoch: 45/50...  Training Step: 126300...  Training loss: 1.5377...  0.0657 sec/batch
Epoch: 45/50...  Training Step: 126350...  Training loss: 1.3661...  0.0640 sec/batch
Epoch: 45/50...  Training Step: 126400...  Training loss: 1.3165...  0.0671 sec/batch
Epoch: 45/50...  Training Step: 126450...  Training loss: 1.3099...  0.0661 sec/batch
Epoch: 45/50...  Training Step: 126500...  Training loss: 1.4387...  0.0661 sec/batch
Epoch: 45/50...  Training Step: 126550...  Training loss: 1.6005...  0.0653 sec/batch
Epoch: 45/50...  Training Step: 126600...  Training loss: 1.3145...  0.0657 sec/batch
Epoch: 45/50...  Training Step: 126650...  Training loss: 1.6387...  0.0650 sec/batch
Epoch: 45/50...  Training Step: 126700...  Training loss: 1.6447...  0.0739 sec/batch
Epoch: 45/50...  Training Step: 126750...  Training lo

Epoch: 47/50...  Training Step: 131000...  Training loss: 1.3940...  0.0729 sec/batch
Epoch: 47/50...  Training Step: 131050...  Training loss: 1.5214...  0.0676 sec/batch
Epoch: 47/50...  Training Step: 131100...  Training loss: 1.2390...  0.0683 sec/batch
Epoch: 47/50...  Training Step: 131150...  Training loss: 1.2922...  0.0660 sec/batch
Epoch: 47/50...  Training Step: 131200...  Training loss: 1.3632...  0.0656 sec/batch
Epoch: 47/50...  Training Step: 131250...  Training loss: 1.3880...  0.0657 sec/batch
Epoch: 47/50...  Training Step: 131300...  Training loss: 1.4356...  0.0646 sec/batch
Epoch: 47/50...  Training Step: 131350...  Training loss: 1.4119...  0.0663 sec/batch
Epoch: 47/50...  Training Step: 131400...  Training loss: 1.2242...  0.0675 sec/batch
Epoch: 47/50...  Training Step: 131450...  Training loss: 1.4017...  0.0662 sec/batch
Epoch: 47/50...  Training Step: 131500...  Training loss: 1.3612...  0.0701 sec/batch
Epoch: 47/50...  Training Step: 131550...  Training lo

Epoch: 49/50...  Training Step: 135800...  Training loss: 1.2066...  0.0655 sec/batch
Epoch: 49/50...  Training Step: 135850...  Training loss: 1.3044...  0.0690 sec/batch
Epoch: 49/50...  Training Step: 135900...  Training loss: 1.4199...  0.0663 sec/batch
Epoch: 49/50...  Training Step: 135950...  Training loss: 1.3181...  0.0651 sec/batch
Epoch: 49/50...  Training Step: 136000...  Training loss: 1.6393...  0.0659 sec/batch
Epoch: 49/50...  Training Step: 136050...  Training loss: 1.3783...  0.0658 sec/batch
Epoch: 49/50...  Training Step: 136100...  Training loss: 1.3599...  0.0653 sec/batch
Epoch: 49/50...  Training Step: 136150...  Training loss: 1.4192...  0.0652 sec/batch
Epoch: 49/50...  Training Step: 136200...  Training loss: 1.6977...  0.0648 sec/batch
Epoch: 49/50...  Training Step: 136250...  Training loss: 1.5010...  0.0649 sec/batch
Epoch: 49/50...  Training Step: 136300...  Training loss: 1.3939...  0.0648 sec/batch
Epoch: 49/50...  Training Step: 136350...  Training lo

Epoch: 50/50...  Training Step: 140600...  Training loss: 1.3842...  0.0680 sec/batch
Epoch: 50/50...  Training Step: 140650...  Training loss: 1.6347...  0.0653 sec/batch
Epoch: 50/50...  Training Step: 140700...  Training loss: 1.1554...  0.0655 sec/batch
Epoch: 50/50...  Training Step: 140750...  Training loss: 1.5034...  0.0654 sec/batch
Epoch: 50/50...  Training Step: 140800...  Training loss: 1.5229...  0.0650 sec/batch
Epoch: 50/50...  Training Step: 140850...  Training loss: 2.0079...  0.0775 sec/batch
Epoch: 50/50...  Training Step: 140900...  Training loss: 1.1776...  0.0689 sec/batch
Epoch: 50/50...  Training Step: 140950...  Training loss: 1.4612...  0.0664 sec/batch
Epoch: 50/50...  Training Step: 141000...  Training loss: 1.3102...  0.0664 sec/batch
Epoch: 50/50...  Training Step: 141050...  Training loss: 1.2856...  0.0654 sec/batch


#### Saved checkpoints

Read up on saving and loading checkpoints here: https://www.tensorflow.org/programmers_guide/variables

In [22]:
tf.train.get_checkpoint_state('checkpoints')

model_checkpoint_path: "checkpoints/i141050_l128.ckpt"
all_model_checkpoint_paths: "checkpoints/i121400_l128.ckpt"
all_model_checkpoint_paths: "checkpoints/i121600_l128.ckpt"
all_model_checkpoint_paths: "checkpoints/i121800_l128.ckpt"
all_model_checkpoint_paths: "checkpoints/i122000_l128.ckpt"
all_model_checkpoint_paths: "checkpoints/i122200_l128.ckpt"
all_model_checkpoint_paths: "checkpoints/i122400_l128.ckpt"
all_model_checkpoint_paths: "checkpoints/i122600_l128.ckpt"
all_model_checkpoint_paths: "checkpoints/i122800_l128.ckpt"
all_model_checkpoint_paths: "checkpoints/i123000_l128.ckpt"
all_model_checkpoint_paths: "checkpoints/i123200_l128.ckpt"
all_model_checkpoint_paths: "checkpoints/i123400_l128.ckpt"
all_model_checkpoint_paths: "checkpoints/i123600_l128.ckpt"
all_model_checkpoint_paths: "checkpoints/i123800_l128.ckpt"
all_model_checkpoint_paths: "checkpoints/i124000_l128.ckpt"
all_model_checkpoint_paths: "checkpoints/i124200_l128.ckpt"
all_model_checkpoint_paths: "checkpoints/i124

## Sampling

Now that the network is trained, we'll can use it to generate new text. The idea is that we pass in a character, then the network will predict the next character. We can use the new one, to predict the next one. And we keep doing this to generate all new text. I also included some functionality to prime the network with some text by passing in a string and building up a state from that.

The network gives us predictions for each character. To reduce noise and make things a little less random, I'm going to only choose a new character from the top N most likely characters.



In [23]:
def pick_top_n(preds, vocab_size, top_n=5):
    p = np.squeeze(preds)
    p[np.argsort(p)[:-top_n]] = 0
    p = p / np.sum(p)
    c = np.random.choice(vocab_size, 1, p=p)[0]
    return c

In [24]:
def sample(checkpoint, n_samples, lstm_size, vocab_size, prime="The "):
    samples = [c for c in prime]
    model = CharRNN(len(vocab), lstm_size=lstm_size, sampling=True)
    saver = tf.train.Saver()
    with tf.Session() as sess:
        saver.restore(sess, checkpoint)
        new_state = sess.run(model.initial_state)
        for c in prime:
            x = np.zeros((1, 1))
            x[0,0] = vocab_to_int[c]
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

        c = pick_top_n(preds, len(vocab))
        samples.append(int_to_vocab[c])

        for i in range(n_samples):
            x[0,0] = c
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

            c = pick_top_n(preds, len(vocab))
            samples.append(int_to_vocab[c])
        
    return ''.join(samples)

Here, pass in the path to a checkpoint and sample from the network.

In [25]:
tf.train.latest_checkpoint('checkpoints')

'checkpoints/i141050_l128.ckpt'

In [26]:
checkpoint = tf.train.latest_checkpoint('checkpoints')
samp = sample(checkpoint, 2000, lstm_size, len(vocab), prime="function")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/i141050_l128.ckpt
function.paiting = function(callback) {
                                                                                                                                                          errsable.complete(socket, callback) {
                                      // a can this this to
                                            canced = this.emit(`${this.docon}`;
                                case 'resolve':
                              if (typeof self.down === 'string') {
                                case CTRAN Servers.prototype.path the root the active sended it a duplem possition.
                      // Options.
                      }
                  } else {
                    throw new ERR_HASTION_OPTION_INGABLE('if');
                }
              }

              this.settoring() => {
                return flush();

            }
            case 'setup':
            this.connected = tru

In [53]:
checkpoint = 'checkpoints/i200_l512.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/i200_l512.ckpt


NotFoundError: Unsuccessful TensorSliceReader constructor: Failed to find any matching files for checkpoints/i200_l512.ckpt
	 [[Node: save/RestoreV2 = RestoreV2[dtypes=[DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, ..., DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT], _device="/job:localhost/replica:0/task:0/device:CPU:0"](_arg_save/Const_0_0, save/RestoreV2/tensor_names, save/RestoreV2/shape_and_slices)]]

Caused by op 'save/RestoreV2', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/siyuan/venv/ml/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/siyuan/venv/ml/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/siyuan/venv/ml/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 

In [54]:
checkpoint = 'checkpoints/i600_l512.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/i600_l512.ckpt


NotFoundError: Unsuccessful TensorSliceReader constructor: Failed to find any matching files for checkpoints/i600_l512.ckpt
	 [[Node: save/RestoreV2 = RestoreV2[dtypes=[DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, ..., DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT], _device="/job:localhost/replica:0/task:0/device:CPU:0"](_arg_save/Const_0_0, save/RestoreV2/tensor_names, save/RestoreV2/shape_and_slices)]]
	 [[Node: save/RestoreV2/_1 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device_incarnation=1, tensor_name="edge_6_save/RestoreV2", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"]()]]

Caused by op 'save/RestoreV2', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/siyuan/venv/ml

In [ ]:
checkpoint = 'checkpoints/i1200_l512.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)